In [2]:
!pip install Pillow

In [22]:
import os
import pathlib

import glob
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import *

In [23]:
def load_images_and_labels(image_paths):
    images = []
    labels = []

    for image_path in image_paths:
        image = load_img(image_path, target_size=(32, 32),
                         color_mode='grayscale')
        image = img_to_array(image)

        label = image_path.split(os.path.sep)[-2]
        label = 'positive' in label
        label = float(label)

        images.append(image)
        labels.append(label)

    return np.array(images), np.array(labels)

In [24]:
def build_network():
    input_layer = Input(shape=(32, 32, 1))
    x = Conv2D(filters=20,
               kernel_size=(5, 5),
               padding='same',
               strides=(1, 1))(input_layer)
    x = ELU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2),
                     strides=(2, 2))(x)
    x = Dropout(0.4)(x)

    x = Conv2D(filters=50,
               kernel_size=(5, 5),
               padding='same',
               strides=(1, 1))(x)
    x = ELU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2),
                     strides=(2, 2))(x)
    x = Dropout(0.4)(x)

    x = Flatten()(x)
    x = Dense(units=500)(x)
    x = ELU()(x)
    x = Dropout(0.4)(x)

    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_layer, outputs=output)
    return model

In [40]:
files_pattern = 'C:/Users/hp/Desktop/simple classifier to detect smile/SMILEsmileD-master/SMILEs/*/*/*.jpg'
files_pattern = str(files_pattern)
dataset_paths = [*glob.glob(files_pattern)]

In [41]:
X, y = load_images_and_labels(dataset_paths)

In [42]:
X /= 255.0
total = len(y)
total_positive = np.sum(y)
total_negative = total - total_positive
print(f'Total images: {total}')
print(f'Smile images: {total_positive}')
print(f'Non-smile images: {total_negative}')

Total images: 13165
Smile images: 3690.0
Non-smile images: 9475.0


In [43]:
(X_train, X_test,
 y_train, y_test) = train_test_split(X, y,
                                     test_size=0.2,
                                     stratify=y,
                                     random_state=999)
(X_train, X_val,
 y_train, y_val) = train_test_split(X_train, y_train,
                                    test_size=0.2,
                                    stratify=y_train,
                                    random_state=999)

In [29]:
model = build_network()
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [44]:
BATCH_SIZE = 32
EPOCHS = 20
model.fit(X_train, y_train,
          validation_data=(X_val, y_val),
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          class_weight={
              1.0: total / total_positive,
              0.0: total / total_negative
          })

Epoch 1/20
264/264 [==============================] - 19s 69ms/step - loss: 1.7940 - accuracy: 0.7458 - val_loss: 3.5159 - val_accuracy: 0.4196
Epoch 2/20
264/264 [==============================] - 17s 63ms/step - loss: 0.9950 - accuracy: 0.8185 - val_loss: 0.3184 - val_accuracy: 0.8904
Epoch 3/20
264/264 [==============================] - 17s 64ms/step - loss: 0.8344 - accuracy: 0.8455 - val_loss: 0.3487 - val_accuracy: 0.8586
Epoch 4/20
264/264 [==============================] - 17s 63ms/step - loss: 0.7564 - accuracy: 0.8589 - val_loss: 0.9051 - val_accuracy: 0.8016
Epoch 5/20
264/264 [==============================] - 17s 66ms/step - loss: 0.6519 - accuracy: 0.8732 - val_loss: 0.3881 - val_accuracy: 0.8567
Epoch 6/20
264/264 [==============================] - 18s 67ms/step - loss: 0.6221 - accuracy: 0.8815 - val_loss: 0.2573 - val_accuracy: 0.9027
Epoch 7/20
264/264 [==============================] - 17s 64ms/step - loss: 0.5696 - accuracy: 0.8903 - val_loss: 0.2589 - val_accuracy: